In [2]:
# Load libraries
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv("train.csv")

print('Dataframe Shape: ', data.shape); print('-'*80, '\n');
print("Dataframe Info: \n"); data.info(verbose=True); print('-'*80, '\n')
data.head(5)

Dataframe Shape:  (69999, 172)
-------------------------------------------------------------------------------- 

Dataframe Info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Data columns (total 172 columns):
 #    Column                    Dtype  
---   ------                    -----  
 0    id                        int64  
 1    circle_id                 int64  
 2    loc_og_t2o_mou            float64
 3    std_og_t2o_mou            float64
 4    loc_ic_t2o_mou            float64
 5    last_date_of_month_6      object 
 6    last_date_of_month_7      object 
 7    last_date_of_month_8      object 
 8    arpu_6                    float64
 9    arpu_7                    float64
 10   arpu_8                    float64
 11   onnet_mou_6               float64
 12   onnet_mou_7               float64
 13   onnet_mou_8               float64
 14   offnet_mou_6              float64
 15   offnet_mou_7              float64
 16   offnet_mou_8              float6

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,total_rech_data_6,total_rech_data_7,total_rech_data_8,max_rech_data_6,max_rech_data_7,max_rech_data_8,count_rech_2g_6,count_rech_2g_7,count_rech_2g_8,count_rech_3g_6,count_rech_3g_7,count_rech_3g_8,av_rech_amt_data_6,av_rech_amt_data_7,av_rech_amt_data_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,arpu_3g_6,arpu_3g_7,arpu_3g_8,arpu_2g_6,arpu_2g_7,arpu_2g_8,night_pck_user_6,night_pck_user_7,night_pck_user_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,7.527,48.58,124.38,1.29,32.24,96.68,2.33,0.00,0.0,0.0,0.00,0.0,0.00,2.23,0.00,0.28,5.29,16.04,2.33,0.00,0.00,0.00,0.00,0.00,0.00,7.53,16.04,2.61,46.34,124.38,1.01,18.75,80.61,0.0,0.00,0.0,0.0,0.0,0.0,0.0,65.09,204.99,1.01,0.0,0.0,0.0,8.20,0.63,0.00,0.38,0.0,0.0,81.21,221.68,3.63,2.43,3.68,7.79,0.83,21.08,16.91,0.00,0.00,0.00,3.26,24.76,24.71,0.00,7.61,0.21,7.46,19.96,14.96,0.0,0.0,0.0,0.0,0.0,0.0,7.46,27.58,15.18,11.84,53.04,40.56,0.0,0.0,0.66,0.0,0.0,0.0,1.11,0.69,0.00,3,2,2,77,65,10,65,65,10,6/22/2014,7/10/2014,8/24/2014,65,65,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,0.00,0.0,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,42.953,0.00,0.00,0.00,0.00,25.99,30.89,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,22.01,29.79,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,30.73,31.66,0.00,0.0,0.0,0.00,30.73,31.66,1.68,19.09,10.53,1.41,18.68,11.09,0.35,1.66,3.40,3.44,39.44,25.03,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,3.44,39.44,25.04,0.0,0.0,0.01,0.0,0.0,0.0,0.00,0.00,0.00,3,4,5,0,145,50,0,145,50,6/12/2014,7/10/2014,8/26/2014,0,0,0,NaN,7/8/2014,NaN,NaN,1.0,NaN,NaN,145.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,Na

### Data Analysis

### Identify columns that have no variance

In [4]:
data_unique_count = data.nunique().sort_values(ascending=False)

# Find columns with all NULL entries
data_unique_count_is_zero = data_unique_count[data_unique_count == 0]
print(data_unique_count_is_zero.count())

0


In [5]:
# Find columns with all same entries
data_unique_count_is_one = data_unique_count[data_unique_count == 1]
print(data_unique_count_is_one.count())

13


In [6]:
data_unique_count_is_one

std_ic_t2o_mou_8        1
std_ic_t2o_mou_7        1
circle_id               1
std_og_t2c_mou_8        1
std_og_t2c_mou_7        1
std_og_t2c_mou_6        1
last_date_of_month_8    1
last_date_of_month_7    1
last_date_of_month_6    1
loc_ic_t2o_mou          1
std_og_t2o_mou          1
loc_og_t2o_mou          1
std_ic_t2o_mou_6        1
dtype: int64

In [7]:
### Drop columns with single value in all rows

drop_no_varience_columns = list(data_unique_count_is_one.index)
data.drop(drop_no_varience_columns, axis=1, inplace=True)

In [8]:
data.shape

(69999, 159)

### Update date fields

In [9]:
# covert date columns to python datetime format
date_vars = ["date_of_last_rech_6", "date_of_last_rech_7", "date_of_last_rech_8", 
             "date_of_last_rech_data_6", "date_of_last_rech_data_7", "date_of_last_rech_data_8"]
data[date_vars].head() 

,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8
0,6/22/2014,7/10/2014,8/24/2014,NaN,NaN,NaN
1,6/12/2014,7/10/2014,8/26/2014,NaN,7/8/2014,NaN
2,6/11/2014,7/22/2014,8/24/2014,NaN,NaN,NaN
3,6/15/2014,7/21/2014,8/25/2014,NaN,NaN,NaN
4,6/25/2014,7/26/2014,8/30/2014,6/25/2014,7/23/2014,8/20/2014


In [10]:
# convert to datetime
for col in date_vars:
    data[col] = pd.to_datetime(data[col])

print(data[date_vars].info())
data[date_vars].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date_of_last_rech_6       68898 non-null  datetime64[ns]
 1   date_of_last_rech_7       68765 non-null  datetime64[ns]
 2   date_of_last_rech_8       67538 non-null  datetime64[ns]
 3   date_of_last_rech_data_6  17568 non-null  datetime64[ns]
 4   date_of_last_rech_data_7  17865 non-null  datetime64[ns]
 5   date_of_last_rech_data_8  18417 non-null  datetime64[ns]
dtypes: datetime64[ns](6)
memory usage: 3.2 MB
None


,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8
0,2014-06-22,2014-07-10,2014-08-24,NaT,NaT,NaT
1,2014-06-12,2014-07-10,2014-08-26,NaT,2014-07-08,NaT
2,2014-06-11,2014-07-22,2014-08-24,NaT,NaT,NaT
3,2014-06-15,2014-07-21,2014-08-25,NaT,NaT,NaT
4,2014-06-25,2014-07-26,2014-08-30,2014-06-25,2014-07-23,2014-08-20


In [11]:
## check if any categorical variables are there
num_cols = data._get_numeric_data().columns
total_cols = data.columns

print((list(set(total_cols) - set(num_cols))))

['date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_6', 'date_of_last_rech_data_8', 'date_of_last_rech_data_6', 'date_of_last_rech_data_7']


In [12]:
# Now let's create new features from date column
data['date_of_last_rech_day_6'] = data['date_of_last_rech_6'].dt.day
data['date_of_last_rech_dayofwk_6'] = data['date_of_last_rech_6'].dt.dayofweek.astype(str)

data['date_of_last_rech_day_7'] = data['date_of_last_rech_7'].dt.day
data['date_of_last_rech_dayofwk_7'] = data['date_of_last_rech_7'].dt.dayofweek.astype(str)

data['date_of_last_rech_day_8'] = data['date_of_last_rech_8'].dt.day
data['date_of_last_rech_dayofwk_8'] = data['date_of_last_rech_8'].dt.dayofweek.astype(str)

data['date_of_last_rech_data_day_6'] = data['date_of_last_rech_data_6'].dt.day
data['date_of_last_rech_data_dayofwk_6'] = data['date_of_last_rech_data_6'].dt.dayofweek.astype(str)

data['date_of_last_rech_data_day_7'] = data['date_of_last_rech_data_7'].dt.day
data['date_of_last_rech_data_dayofwk_7'] = data['date_of_last_rech_data_7'].dt.dayofweek.astype(str)

data['date_of_last_rech_data_day_8'] = data['date_of_last_rech_data_8'].dt.day
data['date_of_last_rech_data_dayofwk_8'] = data['date_of_last_rech_data_8'].dt.dayofweek.astype(str)

In [13]:
# Drop all old date columns: add dates columns to drop_column list
data.drop(date_vars, axis=1, inplace=True)

In [14]:
# Columns with more than 70% missing values

columns_missing_data = round(100*(data.isnull().sum()/len(data.index)), 2)
columns_missing_data[columns_missing_data >= 70]

total_rech_data_6               74.90
total_rech_data_7               74.48
total_rech_data_8               73.69
max_rech_data_6                 74.90
max_rech_data_7                 74.48
max_rech_data_8                 73.69
count_rech_2g_6                 74.90
count_rech_2g_7                 74.48
count_rech_2g_8                 73.69
count_rech_3g_6                 74.90
count_rech_3g_7                 74.48
count_rech_3g_8                 73.69
av_rech_amt_data_6              74.90
av_rech_amt_data_7              74.48
av_rech_amt_data_8              73.69
arpu_3g_6                       74.90
arpu_3g_7                       74.48
arpu_3g_8                       73.69
arpu_2g_6                       74.90
arpu_2g_7                       74.48
arpu_2g_8                       73.69
night_pck_user_6                74.90
night_pck_user_7                74.48
night_pck_user_8                73.69
fb_user_6                       74.90
fb_user_7                       74.48
fb_user_8   

## Missing value handling
## High value customers

## Modelling Start

In [15]:
X = data[data.columns[~data.columns.isin(['churn_probability'])]]
Y = data['churn_probability']

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [17]:
# Splitting the data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_std,Y, train_size=0.7,test_size=0.3,random_state=42)

In [18]:
print("Training dataset size",X_train.shape)
print("Training dataset target size",y_train.shape)
print("Test dataset size",X_test.shape)
print("Test dataset target size",y_test.shape)

Training dataset size (48999, 164)
Training dataset target size (48999,)
Test dataset size (21000, 164)
Test dataset target size (21000,)


In [19]:
print("Data imbalance, counts of label '1': {}".format(sum(y_train==1)))
print("Data imbalance, counts of label '0': {} \n".format(sum(y_train==0)))

Data imbalance, counts of label '1': 4977
Data imbalance, counts of label '0': 44022 



from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_train_smote, y_train_smote= sm.fit_sample(X_train, y_train.ravel())

print('After update, the shape of train_X: {}'.format(X_train_smote.shape))
print('After update, the shape of train_y: {} \n'.format(y_train_smote.shape))

print("After update, counts of label '1': {}".format(sum(y_train_smote==1)))
print("After update, counts of label '0': {}".format(sum(y_train_smote==0)))

#Importing the PCA module
from sklearn.decomposition import PCA
pca = PCA(random_state=42)

#Doing PCA on the train data
pca.fit(X_train)

pca.components_
pca.explained_variance_ratio_
pca.explained_variance_ratio_.cumsum()

#Making the screeplot - plotting the cumulative variance against the number of components
%matplotlib inline
fig = plt.figure(figsize = (12,8))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

#pick the number of components explaining max variance
pca_X= PCA(n_components=X)

df_train_pca_X = pca_X.fit_transform(X_tr)
print(df_train_pca_X.shape)
df_test_pca_X = pca_X.transform(X_test)
print(df_test_pca_X.shape)

Logistic Regression

#Run the model using the selected variables
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
LR = LogisticRegression()
LR.fit(df_train_pca_X, y_tr)

#Predicted probabilities
y_pred_X = LR.predict(df_test_pca_X)

#Printing confusion matrix
print(confusion_matrix(y_test,y_pred_X))
print("LogisticRegression accuracy with PCA: ",accuracy_score(y_test,y_pred_X))

#Random Forest - Default parameters

#Importing random forest classifier from sklearn library
from sklearn.ensemble import RandomForestClassifier

#Running the random forest with default parameters.
rfc = RandomForestClassifier()

#fit
rfc.fit(X_train_smote, y_train_smote)

#Making predictions
X_test = pd.DataFrame(data=X_test).iloc[:, pos]

predictions = rfc.predict(X_test)

#Importing classification report and confusion matrix from sklearn metrics
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

#Let's check the report of our default model
print(classification_report(y_test,predictions))

#Printing confusion matrix
print(confusion_matrix(y_test, predictions))

print(accuracy_score(y_test, predictions))

#Random Forest - Parameter tuning
- max_depth
- min_samples_leaf
- min_samples_split
- n_estimators
- max_features

#sample values
rfc = RandomForestClassifier(bootstrap=True,
                              max_depth=10,
                              min_samples_leaf=100, 
                              min_samples_split=200,
                              max_features=10,
                              n_estimators=100)
							  
#fit
rfc.fit(X_train_smote, y_train_smote)

#Making predictions
X_test = pd.DataFrame(data=X_test).iloc[:, pos]

predictions = rfc.predict(X_test)

#Importing classification report and confusion matrix from sklearn metrics
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

#Let's check the report of our default model
print(classification_report(y_test,predictions))

#Printing confusion matrix
print(confusion_matrix(y_test, predictions))

print(accuracy_score(y_test, predictions))